# Bedrock

```{=mdx}

:::caution
您当前所在的页面是关于将 Amazon Bedrock 模型用作 [文本补全模型](/docs/concepts/text_llms) 的文档。Bedrock 上提供的许多流行模型都是 [聊天补全模型](/docs/concepts/chat_models)。

您可能需要查看 [此页面](/docs/integrations/chat/bedrock/)。
:::

```

> [Amazon Bedrock](https://aws.amazon.com/bedrock/) 是一项完全托管的服务，通过 API 提供来自领先 AI 初创公司和 Amazon 的基础模型 (FM)。
> 您可以从一系列广泛的 FM 中进行选择，以找到最适合您使用场景的模型。

这将帮助您使用 LangChain 开始使用 Bedrock 补全模型 (LLM)。有关 `Bedrock` 功能和配置选项的详细文档，请参阅 [API 参考](https://api.js.langchain.com/classes/langchain_community_llms_bedrock.Bedrock.html)。

## 概述
### 集成详情

| 类 | 包 | 本地 | 可序列化 | [Python 支持](https://python.langchain.com/docs/integrations/llms/bedrock) | 包下载量 | 最新包 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [Bedrock](https://api.js.langchain.com/classes/langchain_community_llms_bedrock.Bedrock.html) | [@langchain/community](https://api.js.langchain.com/modules/langchain_community_llms_bedrock.html) | ❌ | ✅ | ✅ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/community?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 准备工作

要访问 Bedrock 模型，您需要创建一个 AWS 账户，获取 API 密钥，并安装 `@langchain/community` 集成以及一些对等依赖项。

### 凭证

前往 [aws.amazon.com](https://aws.amazon.com) 注册 AWS Bedrock 并生成 API 密钥。完成此操作后，请设置环境变量：

```bash
export BEDROCK_AWS_REGION="your-region-url"
export BEDROCK_AWS_ACCESS_KEY_ID="your-access-key-id"
export BEDROCK_AWS_SECRET_ACCESS_KEY="your-secret-access-key"
```

如果您希望自动追踪您的模型调用，也可以通过取消以下注释来设置 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain Bedrock 集成位于 `@langchain/community` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>

同时安装对等依赖项：

<Npm2Yarn>
  @aws-crypto/sha256-js @aws-sdk/credential-provider-node @smithy/protocol-http @smithy/signature-v4 @smithy/eventstream-codec @smithy/util-utf8 @aws-sdk/types
</Npm2Yarn>

您还可以在 Web 环境（例如 Edge 函数或 Cloudflare Workers）中使用 Bedrock，只需省略 `@aws-sdk/credential-provider-node` 依赖项，并使用 `web` 入口点：

<Npm2Yarn>
  @aws-crypto/sha256-js @smithy/protocol-http @smithy/signature-v4 @smithy/eventstream-codec @smithy/util-utf8 @aws-sdk/types
</Npm2Yarn>

```

## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全结果：

In [ ]:
// @lc-docs-hide-cell
// Deno requires these imports, and way of loading env vars.
// we don't want to expose in the docs.
// Below this cell we have a typescript markdown codeblock with
// the node code.
import "@aws-sdk/credential-provider-node";
import "@smithy/protocol-http";
import "@aws-crypto/sha256-js";
import "@smithy/protocol-http";
import "@smithy/signature-v4";
import "@smithy/eventstream-codec";
import "@smithy/util-utf8";
import "@aws-sdk/types";
import { Bedrock } from "@langchain/community/llms/bedrock"
import { getEnvironmentVariable } from "@langchain/core/utils/env";

const llm = new Bedrock({
  model: "anthropic.claude-v2",
  region: "us-east-1",
  // endpointUrl: "custom.amazonaws.com",
  credentials: {
    accessKeyId: getEnvironmentVariable("BEDROCK_AWS_ACCESS_KEY_ID"),
    secretAccessKey: getEnvironmentVariable("BEDROCK_AWS_SECRET_ACCESS_KEY"),
  },
  temperature: 0,
  maxTokens: undefined,
  maxRetries: 2,
  // other params...
})

```typescript
import { Bedrock } from "@langchain/community/llms/bedrock"

const llm = new Bedrock({
  model: "anthropic.claude-v2",
  region: process.env.BEDROCK_AWS_REGION ?? "us-east-1",
  // endpointUrl: "custom.amazonaws.com",
  credentials: {
    accessKeyId: process.env.BEDROCK_AWS_ACCESS_KEY_ID,
    secretAccessKey: process.env.BEDROCK_AWS_SECRET_ACCESS_KEY,
  },
  temperature: 0,
  maxTokens: undefined,
  maxRetries: 2,
  // 其他参数...
})
```

## 调用

请注意，某些模型需要特定的提示技术。例如，如果提示不以 `Human: ` 开头，Anthropic 的 Claude-v2 模型将会抛出错误。

In [3]:
const inputText = "Human: Bedrock is an AI company that\nAssistant: "

const completion = await llm.invoke(inputText)
completion

" Here are a few key points about Bedrock AI:\n" +
  "\n" +
  "- Bedrock was founded in 2021 and is based in San Fran"... 116 more characters

## 链式调用

我们可以像这样将补全模型与提示模板[链式调用](/docs/how_to/sequence/)：

In [21]:
import { PromptTemplate } from "@langchain/core/prompts"

const prompt = PromptTemplate.fromTemplate("Human: How to say {input} in {output_language}:\nAssistant:")

const chain = prompt.pipe(llm);
await chain.invoke(
  {
    output_language: "German",
    input: "I love programming.",
  }
)

' Here is how to say "I love programming" in German:\n' +
  "\n" +
  "Ich liebe das Programmieren."

## API 参考文档

有关所有 Bedrock 特性和配置的详细文档，请访问 API 参考页面: https://api.js.langchain.com/classes/langchain_community_llms_bedrock.Bedrock.html